## Import Library

In [55]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from IPython.display import display
from IPython.display import Markdown
import requests 
from bs4 import BeautifulSoup

## WebBasedLoader

In [56]:
BASE_URL = "https://brainlox.com"
COURSES_URL = f"{BASE_URL}/courses"

def get_course_links():
    """Scrapes course list page to extract course URLs"""
    response = requests.get(COURSES_URL)
    if response.status_code != 200:
        print(f"Failed to fetch courses. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    courses = []

    for card in soup.find_all('div', class_='single-courses-box'):
        try:
            course_name = card.find('h3').get_text(strip=True)
            course_details = card.find('p').get_text(strip=True)
            details_link = card.find('a', class_="BookDemo-btn", href=True)['href']
            full_url = BASE_URL + details_link

            courses.append({
                'name': course_name,
                'description': course_details,
                'url': full_url
            })
        except AttributeError:
            continue

    return courses

def load_course_data(url):
    """Loads course page data using WebBaseLoader from LangChain"""
    loader = WebBaseLoader(url)
    documents = loader.load()
    return documents

def fetch_all_course_data():
    """Fetches all course details and loads page content"""
    courses = get_course_links()
    for course in courses:
        print(f"Fetching data for: {course['name']} ({course['url']})")
        course['content'] = load_course_data(course['url'])
    
    return courses

# Run script
courses_data = fetch_all_course_data()

Fetching data for: LEARN SCRATCH PROGRAMING (https://brainlox.com/courses/4f629d96-5ed9-4302-ae0e-3479c543a49e)
Fetching data for: LEARN CLOUD COMPUTING BASICS-AWS (https://brainlox.com/courses/872d1cb6-8469-4797-b267-8c41837b10e2)
Fetching data for: LEARN MOBILE DEVELOPMENT (https://brainlox.com/courses/2cf11f62-6452-41f1-9b42-303fb371b873)
Fetching data for: LEARN CORE JAVA PROGRAMMING ONLINE (https://brainlox.com/courses/be32e068-edca-4b41-96ee-4839de6aaebb)
Fetching data for: LEARN ROBOTICS (https://brainlox.com/courses/0deafb39-3208-42db-93e3-bd69f8562f82)
Fetching data for: LEARN JAVASCRIPT (https://brainlox.com/courses/fc9e2faf-dbe1-47bf-994c-f566a9ad3b42)
Fetching data for: LEARN NODE JS (https://brainlox.com/courses/b0f2428a-c1c0-4def-8ac2-692a2d51a5b4)
Fetching data for: LEARN CLOUD COMPUTING ADVANCE-AWS (https://brainlox.com/courses/e0edfcf8-9e0f-4c7a-bc90-3000822924e2)
Fetching data for: PYTHON PROGRAMMING-BEGINNER (https://brainlox.com/courses/fc29b015-962f-41fc-bc93-181d3

In [57]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


FAISS_INDEX_PATH = "faiss_courses_index"

def create_vector_store(courses):
    """Converts course data to embeddings and stores in FAISS"""
    embeddings = OllamaEmbeddings(model="llama3.2:latest")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

    # Convert course data into LangChain Document format
    docs = []
    for course in courses:
        content_text = course['content'][0].page_content if course['content'] else ""
        doc = Document(
            page_content=f"Course Name: {course['name']}\nDescription: {course['description']}\n{content_text}",
            metadata={"url": course["url"]}
        )
        docs.append(doc)

    # Split long documents
    split_docs = text_splitter.split_documents(docs)

    # Create or Load FAISS vector store
    if os.path.exists(FAISS_INDEX_PATH):
        vector_store = FAISS.load_local(FAISS_INDEX_PATH, embeddings)
        print("Loaded existing FAISS index.")
    else:
        vector_store = FAISS.from_documents(split_docs, embeddings)
        vector_store.save_local(FAISS_INDEX_PATH)
        print("Created and saved new FAISS index.")

    return vector_store

# Run storage function
vector_store = create_vector_store(courses_data)
print("✅ Courses stored in FAISS successfully!")

Created and saved new FAISS index.
✅ Courses stored in FAISS successfully!


## Prompt Tempelate

In [58]:
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_template("""
# Answer the following question based only on the provided context.
# Thing step by step before providing a detailed answer.
# I will tip you $1000 if the use find the answer helpful.
# <context>
# {context}
# </context>
# Question: {input}
# """)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are an AI chatbot specializing in answering questions about technical courses.
Use only the provided context to generate informative, precise, and well-structured responses.
Think step by step before providing a detailed answer.
If the user finds your response helpful, a $1000 tip is promised.

<context>
{context}
</context>

Question: {input}

Provide a clear, concise, and relevant answer based on the given context.
""")


In [59]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2:latest")
llm


## Chain Introduction
## Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

documents_chain = create_stuff_documents_chain(llm, prompt)


In [60]:
"""A retriever is an interface that returns documents given an unstructured query.
 It is more general than a vector store. A retriever does not need to be able to store
 documents, only to return (or retrieve) them. Vector stores can be used as
   the backbone of a retriever, but there are other types of retrievers as well."""


retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FF214A6840>, search_kwargs={})

In [61]:
#How to create
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, documents_chain)

In [70]:
responce = retrieval_chain.invoke({"input":"details LEARN CORE JAVA PROGRAMMING ONLINE"})

In [71]:
(responce['answer'])

'Based on the provided context, here\'s a detailed response:\n\nThe LEARN CORE JAVA PROGRAMMING ONLINE course is designed to introduce students to the basics of Java programming. As stated in the course description, "Java is a very popular high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible." This means that Java applications can run on any platform that supports Java without requiring recompilation.\n\nThe course covers various topics, including:\n\n* Introduction to Java programming and its relation to Minecraft\n* Advanced topics in Minecraft Java programming\n\nThroughout the six-week course, students will learn the basics of Java programming while creating Minecraft mods and plugins. This hands-on approach allows learners to apply theoretical knowledge to real-world projects, making the learning experience more engaging and effective.\n\nBy the end of this course, students can expect to gain a solid 

In [72]:
import textwrap
def to_markdown(text):
    text = text.replace("*", " *")
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _:True))
to_markdown(responce['answer'])

> Based on the provided context, here's a detailed response:
> 
> The LEARN CORE JAVA PROGRAMMING ONLINE course is designed to introduce students to the basics of Java programming. As stated in the course description, "Java is a very popular high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible." This means that Java applications can run on any platform that supports Java without requiring recompilation.
> 
> The course covers various topics, including:
> 
>  * Introduction to Java programming and its relation to Minecraft
>  * Advanced topics in Minecraft Java programming
> 
> Throughout the six-week course, students will learn the basics of Java programming while creating Minecraft mods and plugins. This hands-on approach allows learners to apply theoretical knowledge to real-world projects, making the learning experience more engaging and effective.
> 
> By the end of this course, students can expect to gain a solid understanding of Java programming principles and be able to create their own Minecraft mods and plugins.
> 
> Would you like to know more about Java or is there something specific you'd like me to elaborate on?